In [1]:
from Common import *
from peewee import *
from logger import *
from models import *  
import xml.etree.ElementTree as ET
import subprocess
import os.path
logger = get_logger(path = 'test_sdmx.log')


In [2]:
import time
######### leggo i risultati dello scraping e seleziono le nuove osservazioni da scaricare
    ######### Aggiunto scarico + parsing delle constraint + individuazione livello geografico 
        ######### aggiunta modifica dei campi Download e Volume post scarico
            ###### gestito parametro frequenza in base alle constraint(si provano tutte), **ELIMINATO** frequenza=annuale
                ######## TODO: gestire scarichi multipli in base ai livelli geografici minimi disponibili
                                # Al momento scarica al livello geo minimo disponibile
                    ###### Aggiunto controllo sulla pre-esistenza del file constraint.xml


logger.info('**** FASE DOWNLOAD ****')

   
try:
    cur=db.cursor()
    cur.execute('select * from monitor where download=False')
    result=cur.fetchall()
    db.close()

    for flow in result:
        
        flowRef=flow[3]
        start=flow[5]
        end=flow[6]+1
        titolo=flow[2]
        data_agg=flow[4]
        anni=range(start,end)


        logger.info(f'______________{flow[2]}______________')
        logger.info('________________________________________________________')
        logger.info('Controllo constraint in corso')
        
        exist=os.path.isfile(f'./constraint/constraint_{flowRef}.xml') 
        
        if exist:

            constraint=parse_constraint(flowRef)

        else:
            
            get_constraint(flowRef) 
        
            constraint=parse_constraint(flowRef)
              

        geo=get_geo_level(constraint) 
        
        if min_geo_level(geo):

            
            logger.info(f'livello geografico minimo: {geo}. \b download in corso  ')

            #for fr in constraint['FREQ']:
            for a in anni:

                try:
                    
                    cmd=f'Rscript get_data.R -r {flowRef} -f A -s {a} -e {a} -g {geo} -d {down_dir}'
                    print(cmd)

                    result=subprocess.run(cmd, stderr=subprocess.PIPE,stdout=subprocess.PIPE, text=True)
                    volume,note= get_cmd_info(result)
                    logger.info(f'scaricati {volume} in {time}')
                    cur=db.cursor()
                    
                    query=f'insert INTO Download (titolo,flow,data_aggiornamento,anno,volume,note,livello_geografico)values ("{titolo}","{flowRef}","{data_agg}","{a}","{volume}","{note}","{geo}");'
                    cur.execute(query)  
                    logger.info(f'Aggiornata osservazione in db')
                    db.close()
                
                except Exception as e:

                    logger.error(e)
            
            query=f'UPDATE Monitor SET download=True WHERE flow="{flowRef}" and data_aggiornamento="{data_agg}";'            
            cur=db.cursor()
            cur.execute(query)  
            db.close()

        else:

            logger.info(f'livello geografico: {geo}. \b non verrà eseguito il download')
            
            try:
                cur=db.cursor()
                    
                query=f'UPDATE Monitor SET  note = "Livello geografico {geo}" , download=True WHERE flow="{flowRef}" and data_aggiornamento="{data_agg}";'
                cur.execute(query)  
                db.close()
            
            except Exception as e:

                logger.error(e)
    
    logger.info('*** stop***')   
except Exception as e:

    logger.error(e)



[2022-01-24 10:55:46,408]:[sdmx_istat]:[<module>]:[INFO]: **** FASE DOWNLOAD ****
[2022-01-24 10:55:46,436]:[sdmx_istat]:[<module>]:[INFO]: ______________Stranieri residenti al 1° gennaio - Cittadinanza______________
[2022-01-24 10:55:46,436]:[sdmx_istat]:[<module>]:[INFO]: ________________________________________________________
[2022-01-24 10:55:46,437]:[sdmx_istat]:[<module>]:[INFO]: Controllo constraint in corso
[2022-01-24 10:55:46,448]:[sdmx_istat]:[<module>]:[INFO]: livello geografico minimo: Comunale. download in corso  


Rscript get_data.R -r 29_317 -f A -s 2003 -e 2003 -g Comunale


[2022-01-24 10:56:27,665]:[sdmx_istat]:[<module>]:[INFO]: scaricati None in Not Found (HTTP 404).Errore in readSDMX(providerId = ISTAT, resource = data, flowRef = opt$flowref,  : 
  HTTP request failed with status: 404 
Chiamate: %>% -> as.data.frame -> readSDMX
Esecuzione interrotta

[2022-01-24 10:56:27,711]:[sdmx_istat]:[<module>]:[INFO]: Aggiornata osservazione in db


Rscript get_data.R -r 29_317 -f A -s 2004 -e 2004 -g Comunale


[2022-01-24 10:57:07,736]:[sdmx_istat]:[<module>]:[INFO]: scaricati None in Not Found (HTTP 404).Errore in readSDMX(providerId = ISTAT, resource = data, flowRef = opt$flowref,  : 
  HTTP request failed with status: 404 
Chiamate: %>% -> as.data.frame -> readSDMX
Esecuzione interrotta

[2022-01-24 10:57:07,795]:[sdmx_istat]:[<module>]:[INFO]: Aggiornata osservazione in db


...

[2022-01-24 11:04:23,380]:[sdmx_istat]:[<module>]:[INFO]: scaricati None in Not Found (HTTP 404).Errore in readSDMX(providerId = ISTAT, resource = data, flowRef = opt$flowref,  : 
  HTTP request failed with status: 404 
Chiamate: %>% -> as.data.frame -> readSDMX
Esecuzione interrotta

[2022-01-24 11:04:23,411]:[sdmx_istat]:[<module>]:[INFO]: Aggiornata osservazione in db


Rscript get_data.R -r 29_317 -f A -s 2016 -e 2016 -g Comunale


[2022-01-24 11:05:05,949]:[sdmx_istat]:[<module>]:[INFO]: scaricati None in Not Found (HTTP 404).Errore in readSDMX(providerId = ISTAT, resource = data, flowRef = opt$flowref,  : 
  HTTP request failed with status: 404 
Chiamate: %>% -> as.data.frame -> readSDMX
Esecuzione interrotta

[2022-01-24 11:05:06,003]:[sdmx_istat]:[<module>]:[INFO]: Aggiornata osservazione in db


Rscript get_data.R -r 29_317 -f A -s 2017 -e 2017 -g Comunale


[2022-01-24 11:05:46,593]:[sdmx_istat]:[<module>]:[INFO]: scaricati None in Not Found (HTTP 404).Errore in readSDMX(providerId = ISTAT, resource = data, flowRef = opt$flowref,  : 
  HTTP request failed with status: 404 
Chiamate: %>% -> as.data.frame -> readSDMX
Esecuzione interrotta

[2022-01-24 11:05:46,648]:[sdmx_istat]:[<module>]:[INFO]: Aggiornata osservazione in db
[2022-01-24 11:05:46,693]:[sdmx_istat]:[<module>]:[INFO]: *** stop***


In [3]:
from conf import conf

In [4]:
def get_cmd_info(result):
    import re
    size=re.compile('(?:SIZE:)(.*)"')
    sec=re.compile('(.* sec) elapsed')
    err=re.compile('Not Found (HTTP 404)')

    try:
        volume=size.findall(result)[0]
        time=sec.findall(result)[0]
        err=None
    except:
        volume=None
        time=None
        err=err.search(result)

    return volume,time,err

CompletedProcess(args='Rscript get_data.R -r 29_317 -f A -s 2017 -e 2017 -g Comunale', returncode=1, stdout="[rsdmx][INFO] Fetching 'http://sdmx.istat.it/SDMXWS/rest/dataflow/all/all/latest/' \n[rsdmx][INFO] Fetching 'http://sdmx.istat.it/SDMXWS/rest/datastructure/all/DCIS_POPSTRCIT1/latest/?references=children' \n[rsdmx][INFO] Fetching 'http://sdmx.istat.it/SDMXWS/rest/data/29_317/A...037001+037002+037003+037005+037006+037007+037008+037009+037010+037011+037012+037013+037014+037015+037016+037017+037019+037020+037021+037022+037024+037025+037026+037027+037028+037030+037031./all/?startPeriod=2017&endPeriod=2017' \n", stderr='\nCaricamento pacchetto: \'dplyr\'\n\nI seguenti oggetti sono mascherati da \'package:stats\':\n\n    filter, lag\n\nI seguenti oggetti sono mascherati da \'package:base\':\n\n    intersect, setdiff, setequal, union\n\nMessaggio di avvertimento:\nil pacchetto \'optparse\' è stato creato con R versione 4.1.2 \nNot Found (HTTP 404).Errore in readSDMX(providerId = "ISTAT

In [11]:
cur=db.cursor()
cur.execute('select * from monitor where download=False')
result=cur.fetchall()
db.close()
flowRef=flow[3]
start=flow[5]
end=flow[6]
titolo=flow[2]
data_agg=flow[4]
anni=range(start,end)

In [15]:
result=subprocess.run(cmd, stderr=subprocess.PIPE,stdout=subprocess.PIPE, text=True)

In [14]:
cmd='Rscript get_data.R -r 29_317 -f A -s 2021 -e 2021 -g Comunale'

In [13]:
start=flow[5]
end=flow[6]+1
anni=range(start,end)

for a in anni:
    print(a)

2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
